# Access S2S api directly

This notebook walks through an example of sending an API call directly without using the space2stats python library

In [9]:
from typing import Dict

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from geojson_pydantic import Feature, Polygon
from lonboard import Map, ScatterplotLayer
from shapely import from_geojson

In [10]:
BASE_URL = "http://localhost:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [11]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['sum_pop_2020', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020', 'sum_pop_m_2020', 'sum_pop_f_2020']


In [12]:
AOIModel = Feature[Polygon, Dict]

# ~kenya
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [33.78593974945852, 5.115816884114494],
                [33.78593974945852, -4.725410543134203],
                [41.94362577283266, -4.725410543134203],
                [41.94362577283266, 5.115816884114494],
                [33.78593974945852, 5.115816884114494],
            ]
        ],
    },
    "properties": {"name": "Updated AOI"},
}


feat = AOIModel(**aoi)

In [13]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "touches",
    "fields": ["sum_pop_2020"],
    "geometry": "point",
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)

In [14]:
df

,hex_id,geometry,sum_pop_2020
0,866a4a00fffffff,"{""type"":""Point"",""coordinates"":[36.180976490229...",476.538185
1,866a4a017ffffff,"{""type"":""Point"",""coordinates"":[36.078694056627...",676.912804
2,866a4a01fffffff,"{""type"":""Point"",""coordinates"":[36.134817927246...",347.182722
3,866a4a047ffffff,"{""type"":""Point"",""coordinates"":[36.283201202805...",380.988678
4,866a4a04fffffff,"{""type"":""Point"",""coordinates"":[36.339286712001...",285.943490
...,...,...,...
26184,86969ed97ffffff,"{""type"":""Point"",""coordinates"":[34.808746693532...",196.448391
26185,86969ed9fffffff,"{""type"":""Point"",""coordinates"":[34.750412026186...",296.740893
26186,86969eda7ffffff,"{""type"":""Point"",""coordinates"":[34.770464787099...",154.818453
26187,86969edafffffff,"{""type"":""Point"",""coordinates"":[34.712102323801...",157.875350


In [15]:
df["geometry"] = df["geometry"].apply(lambda geom: from_geojson(geom))
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
gdf

,hex_id,geometry,sum_pop_2020
0,866a4a00fffffff,POINT (36.18098 5.12501),476.538185
1,866a4a017ffffff,POINT (36.07869 5.11252),676.912804
2,866a4a01fffffff,POINT (36.13482 5.09047),347.182722
3,866a4a047ffffff,POINT (36.2832 5.13748),380.988678
4,866a4a04fffffff,POINT (36.33929 5.11542),285.943490
...,...,...,...
26184,86969ed97ffffff,POINT (34.80875 -3.47225),196.448391
26185,86969ed9fffffff,POINT (34.75041 -3.44944),296.740893
26186,86969eda7ffffff,POINT (34.77046 -3.5715),154.818453
26187,86969edafffffff,POINT (34.7121 -3.54868),157.875350


In [16]:
# Define custom breaks and corresponding RGBA colors
breaks = [
    gdf["sum_pop_2020"].min(),
    1,
    1000,
    10000,
    50000,
    100000,
    200000,
    gdf["sum_pop_2020"].max(),
]
colors = np.array(
    [
        [211, 211, 211, 255],  # Light gray for 0
        [255, 255, 0, 255],  # Yellow for 1-1000
        [255, 165, 0, 255],  # Orange for 1000-10000
        [255, 0, 0, 255],  # Red for 10000-50000
        [128, 0, 128, 255],  # Purple for 50000-100000
        [0, 0, 255, 255],  # Blue for 100000-200000
        [0, 0, 139, 255],  # Dark blue for 200000+
    ]
)

# Function to assign colors based on custom bins


def assign_color(value, breaks, colors):
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i]
    return colors[-1]  # In case value exceeds all breaks


# Map sum_pop_2020 values to colors using the custom function
gdf["color"] = gdf["sum_pop_2020"].apply(lambda x: assign_color(x, breaks, colors))
colors = np.uint8(gdf["color"].tolist())

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=colors)

m = Map(layer)
m

Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x1411fada0>
[
  [
    2…